# Trabajo Práctico Integrador 1

## Análisis de datos

#### Integrantes
- Daniel Bazan
- Santiago Casado

##| 1. Dataset seleccionado

### **Conflictos armados en ciudades (Cities and Armed Conflict Events, CACE)**

** explicar el dataset **

** info para descargar desde la api

In [47]:
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, MetaData, text
import json
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

#### a. Descargar el dataset a través de la página oficial

Observando el archivo, se tuvo que agregar el delimitador ";" para poder importarlo correctamente.

In [48]:
path = "dataset/CACE_1989-2017.csv"

# df = pd.read_csv(path, delimiter=';', low_memory=False)
df = pd.read_csv(path, delimiter=';')

/var/folders/_f/7gqqb6190c9b0h_7dkhypsj80000gn/T/ipykernel_1326/2020675959.py:4: DtypeWarning: Columns (8,15,17,18,19,20,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=';')


## 2. Pre-procesamiento de los datos

#### a. Columnas y tipos de datos

In [49]:
df.columns

Index(['id', 'active_year', 'type_of_violence', 'conflict_new_id',
       'conflict_name', 'dyad_new_id', 'dyad_name', 'side_a_new_id', 'gwnoa',
       'side_a', 'side_b_new_id', 'gwnob', 'side_b', 'where_prec',
       'where_coordinates', 'City', 'CACE cityname', 'Capital', 'Majorcity',
       'Top3cities', 'Comment', 'adm_1', 'adm_2', 'latitude', 'longitude',
       'geom_wkt', 'priogrid_gid', 'country', 'country_id', 'region',
       'event_clarity', 'date_prec', 'year', 'date_start', 'date_end',
       'deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown', 'best',
       'low', 'high', 'Unnamed: 42', 'Unnamed: 43'],
      dtype='object')

#### b. Detalles de las columnas del dataset

**Fecha**
| Columna       | Descripción                                                |
| ------------- | ---------------------------------------------------------- |
| `active_year` | Estado del conflicto en ese año (1 = activo, 0 = no activo) |
| `year`        | Año del evento |
| `date_start`  | Fecha de inicio del evento |
| `date_end`    | Fecha de fin del evento |
| `date_prec`   | Precisión de la fecha (1 = exacta, 5 = mas de un mes, menos de un año)|

**Ubicacion**
| Columna             | Descripción                                                          |
| ------------------- | -------------------------------------------------------------------- |
| `where_prec`        | Precisión de la ubicación geográfica (1 = exacta, 3 = incierta) |
| `where_coordinates` | Nombre del lugar del evento |
| `adm_1`             | División administrativa de primer nivel (estado, provincia, etc) |
| `adm_2`             | División administrativa de segundo nivel (municipio, distrito, etc)|
| `latitude`          | Latitud del evento |
| `longitude`         | Longitud del evento |
| `geom_wkt`          | Representación geográfica en formato WKT|
| `priogrid_gid`      | ID de celda espacial de 0.5° x 0.5° (Priogrid)|
| `country`           | Nombre del país |
| `country_id`        | Código del país según Gleditsch & Ward |
| `region`            | Región geográfica amplia (America, Asia, etc.)|
| `City`              | 1 = ocurrió en ciudad (>100.000 hab.), 0 = no |
| `CACE cityname`     | Nombre de la ciudad si es evento urbano según ONU |
| `Capital`           | 1 = ocurrió en capital nacional |
| `Majorcity`         | 1 = ocurrió en ciudad de más de 750.000 habitantes |
| `Top3cities`        | Indica si ocurrió en una de las tres ciudades más grandes del país|

**Entidades participantes**
| Columna            | Descripción                                                     |
| ------------------ | --------------------------------------------------------------- |
| `type_of_violence` | Tipo de violencia: 1 = estatal, 2 = no estatal, 3 = unilateral  |
| `conflict_new_id`  | ID conflicto |
| `conflict_name`    | Nombre del conflicto |
| `dyad_new_id`      | ID de la díada (relación entre dos actores)|
| `dyad_name`        | Par de actores que interactúan entre sí|
| `side_a_new_id`    | ID del actor A |
| `gwnoa`            | Código de país del actor A|
| `side_a`           | Nombre del actor que inicia el evento|
| `side_b_new_id`    | ID del actor B|
| `gwnob`            | Código de país del actor B|
| `side_b`           | Nombre del actor que es atacado (puede ser vacío)|

**Víctimas**
| Columna            | Descripción                                          |
| ------------------ | ---------------------------------------------------- |
| `deaths_a`         | Muertes en el actor A |
| `deaths_b`         | Muertes en el actor B |
| `deaths_civilians` | Muertes de civiles |
| `deaths_unknown`   | Muertes no clasificadas |
| `best`             | Estimación central de muertes totales |
| `low`              | Estimación mínima |
| `high`             | Estimación máxima |

**Datos prescindibles**
| Columna         | Descripción                                                            |
| --------------- | ---------------------------------------------------------------------- |
| `id`            | ID general del evento |
| `event_clarity` | Grado de certeza del evento (1 = claro, 3 = dudoso) |
| `Comment`       | Comentarios |
| `Unnamed: 42`   | Columna sin nombre |
| `Unnamed: 43`   | Columna sin nombre |


#### c. Eliminación de columnas

Se eliminan las columnas con información redundante o que no aporten valor para un modelo de machine learning.

In [50]:
columnasPrescindibles = [
    'id',                     
    'conflict_name',          # Redundante con side_a, side_b y dyad
    'side_a_new_id',          
    'side_b_new_id',
    'gwnoa',                  # Redundante con country
    'gwnob',
    'country_id',             
    'geom_wkt',               # Para la ubicación se utilizará la longitud y latitud
    'where_coordinates',      # Texto libre, no aporta valor
    'Comment',                # Taxto libre, no aporta valor
    'Unnamed: 42',            # Columnas vacías
    'Unnamed: 43',
    'event_clarity',          # Precisión de claridad de evento, no ayuda a la predicción
    'date_prec',              # Precisión de fecha, no ayuda a la predicción
    'where_prec',             # Precisión de ubicación, no ayuda a la predicción
    'conflict_new_id',        
    'dyad_new_id',            
    'priogrid_gid'            # Para la ubicación se utilizará la longitud y latitud
]

df_cace = df.drop(columns=columnasPrescindibles, axis = 1)

#### d. Datos faltantes o duplicados

In [51]:
print(f"Valores faltantes: {df_cace.isna().sum()}")  # Calcular valores faltantes

Valores faltantes: active_year              0
type_of_violence         0
dyad_name                0
side_a                   0
side_b                   0
City                     0
CACE cityname       117996
Capital                  0
Majorcity                0
Top3cities               0
adm_1                 7237
adm_2                37225
latitude                 0
longitude                0
country                  0
region                   0
year                     0
date_start               0
date_end                 0
deaths_a                 0
deaths_b                 0
deaths_civilians         0
deaths_unknown           0
best                     0
low                      0
high                     0
dtype: int64


In [52]:
# Calcular la proporción de valores nulos en cada columna
df_cace.isna().mean().round(2)*100

active_year          0.0
type_of_violence     0.0
dyad_name            0.0
side_a               0.0
side_b               0.0
City                 0.0
CACE cityname       83.0
Capital              0.0
Majorcity            0.0
Top3cities           0.0
adm_1                5.0
adm_2               26.0
latitude             0.0
longitude            0.0
country              0.0
region               0.0
year                 0.0
date_start           0.0
date_end             0.0
deaths_a             0.0
deaths_b             0.0
deaths_civilians     0.0
deaths_unknown       0.0
best                 0.0
low                  0.0
high                 0.0
dtype: float64

In [53]:
print("Cantidad de datos duplicados:", df_cace.duplicated().sum())

Cantidad de datos duplicados: 1389


Se eliminan los datos duplicados:

In [54]:
df_cace = df_cace.drop_duplicates()

print("Cantidad de datos duplicados:", df_cace.duplicated().sum())

Cantidad de datos duplicados: 0


Finalmente, el dataset tiene la siguiente dimensión:

In [60]:
df_cace.shape

(141513, 26)

#### e. Correción de tipo de datos

Se identifican las columnas con distintos tipo de datos y se unifica:

In [55]:
columnasMuchosTipos = []

i = 0
for col in df_cace.columns:
    unique_types = df_cace[col].apply(type).unique()
    if len(unique_types) > 1:
        columnasMuchosTipos.append(col)
        print(f"\n Columna {i}: {col}")
        print("Tipos encontrados:", unique_types)
        
        #Se muestran 2 ejemplos para cada columna con mas de un tipo de dato
        for dtype in unique_types:
            examples = df_cace[df_cace[col].apply(lambda x: type(x) == dtype)][col].head(2).tolist()
            print(f"  - Tipo {dtype}: {examples}")

    i += 1


 Columna 5: City
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 1]
  - Tipo <class 'str'>: ['0', '0']

 Columna 6: CACE cityname
Tipos encontrados: [<class 'str'> <class 'float'>]
  - Tipo <class 'str'>: ['Şan‘ā’', '‘Adan ']
  - Tipo <class 'float'>: [nan, nan]

 Columna 7: Capital
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 0]
  - Tipo <class 'str'>: ['0', '0']

 Columna 8: Majorcity
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 0]
  - Tipo <class 'str'>: ['0', '0']

 Columna 9: Top3cities
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 0]
  - Tipo <class 'str'>: ['0', '0']

 Columna 10: adm_1
Tipos encontrados: [<class 'str'> <class 'float'>]
  - Tipo <class 'str'>: ['Am?nat al ‘??imah governorate', '‘Adan governorate']
  - Tipo <class 'float'>: [nan, nan]

 Columna 11: adm_2
Tipos encontrados: [<class 'float'> <class 'str'>]
  - Tipo <class 'float'>: [nan, nan]

In [ ]:
import datetime

# Columnas binarias
binary_cols = ['City', 'Capital', 'Majorcity', 'Top3cities']

for col in binary_cols:
    # Reemplazar valores conocidos
    df_cace[col] = df_cace[col].replace({
        '0': 0, '1': 1,
        0: 0, 1: 1,
        'True': 1, 'False': 0,
        True: 1, False: 0
    })
    
    # Convertir otros valores a NaN
    df_cace[col] = pd.to_numeric(df_cace[col], errors='coerce')
    
    # Rellenar NaN con 0 o 1 según sea apropiado
    df_cace[col] = df_cace[col].fillna(0).astype('int8')

# Columnas de texto
text_cols = ['side_a', 'side_b', 'CACE cityname', 'adm_1', 'adm_2', 'country', 'region']
for col in text_cols:
    df_cace[col] = df_cace[col].fillna('').astype(str)

# Columnas de fechas
date_cols = ['date_start', 'date_end']
for col in date_cols:
    df_cace[col] = pd.to_datetime(df_cace[col])

print("\nTipos unificados:")
print(df_cace.dtypes)


Tipos unificados:
active_year                  int64
type_of_violence             int64
dyad_name                   object
side_a                      object
side_b                      object
City                          int8
CACE cityname               object
Capital                       int8
Majorcity                     int8
Top3cities                    int8
adm_1                       object
adm_2                       object
latitude                   float64
longitude                  float64
country                     object
region                      object
year                         int64
date_start          datetime64[ns]
date_end            datetime64[ns]
deaths_a                     int64
deaths_b                     int64
deaths_civilians             int64
deaths_unknown               int64
best                         int64
low                          int64
high                         int64
dtype: object


#### f. Información general de dataset

In [61]:
df_cace.head()

,active_year,type_of_violence,dyad_name,side_a,side_b,City,CACE cityname,Capital,Majorcity,Top3cities,...,year,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,low,high
0,1,1,Government of Yemen (North Yemen) - AQAP,Government of Yemen (North Yemen),AQAP,1,Şan‘ā’,1,1,1,...,2010,2010-09-25,2010-09-25,2,0,0,0,2,2,2
1,1,3,Government of Yemen (North Yemen) - Civilians,Government of Yemen (North Yemen),Civilians,1,‘Adan,0,0,0,...,2011,2011-02-19,2011-02-19,0,0,0,0,0,0,2
2,1,1,Government of Yemen (North Yemen) - AQAP,Government of Yemen (North Yemen),AQAP,0,,0,0,0,...,2011,2011-04-16,2011-04-16,0,0,0,0,0,0,1
3,1,1,Government of Yemen (North Yemen) - AQAP,Government of Yemen (North Yemen),AQAP,0,,0,0,0,...,2012,2012-06-04,2012-06-04,4,1,0,0,5,5,5
4,1,1,Government of Yemen (North Yemen) - AQAP,Government of Yemen (North Yemen),AQAP,0,,0,0,0,...,2012,2012-12-09,2012-12-09,0,4,1,0,5,5,5


In [62]:
df_cace.info()

<class 'pandas.core.frame.DataFrame'>
Index: 141513 entries, 0 to 142901
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   active_year       141513 non-null  int64         
 1   type_of_violence  141513 non-null  int64         
 2   dyad_name         141513 non-null  object        
 3   side_a            141513 non-null  object        
 4   side_b            141513 non-null  object        
 5   City              141513 non-null  int8          
 6   CACE cityname     141513 non-null  object        
 7   Capital           141513 non-null  int8          
 8   Majorcity         141513 non-null  int8          
 9   Top3cities        141513 non-null  int8          
 10  adm_1             141513 non-null  object        
 11  adm_2             141513 non-null  object        
 12  latitude          141513 non-null  float64       
 13  longitude         141513 non-null  float64       
 14  country  

#### g. Consistencia de datos

In [63]:
# Se verifica si existen valores negativos en features que deberían tener enteros no negativos.

columnasVictimas = ['deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown', 'best', 'low', 'high']
negativos = df_cace[columnasVictimas] < 0
print("Valores negativos por columna:")
print(negativos.sum())

# Verificar fechas: inicio <= fin
fechas_invalidas = df_cace['date_start'] > df_cace['date_end']
print('')
print(f"Eventos con fecha de inicio posterior a la fecha de fin: {fechas_invalidas.sum()}")

# Verificar coordenadas válidas
coordenadasInvalidas = ((df_cace['latitude'] < -90) | (df_cace['latitude'] > 90) |
                         (df_cace['longitude'] < -180) | (df_cace['longitude'] > 180))
print('')
print(f"Coordenadas inválidas: {coordenadasInvalidas.sum()}")

Valores negativos por columna:
deaths_a            0
deaths_b            0
deaths_civilians    0
deaths_unknown      0
best                0
low                 0
high                0
dtype: int64

Eventos con fecha de inicio posterior a la fecha de fin: 0

Coordenadas inválidas: 0


#### h. Ajuste de los tipos de datos

In [64]:
# Categorías nominales
nominales = ['dyad_name', 'side_a', 'side_b', 'CACE cityname', 'adm_1', 'adm_2', 'country', 'region', 'type_of_violence']
df_cace[nominales] = df_cace[nominales].astype('category')

# Categorías ordinales
ordinales = ['City', 'Capital', 'Majorcity', 'Top3cities', 'active_year']
df_cace[ordinales] = df_cace[ordinales].astype('category')

df_cace.info()

<class 'pandas.core.frame.DataFrame'>
Index: 141513 entries, 0 to 142901
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   active_year       141513 non-null  category      
 1   type_of_violence  141513 non-null  category      
 2   dyad_name         141513 non-null  category      
 3   side_a            141513 non-null  category      
 4   side_b            141513 non-null  category      
 5   City              141513 non-null  category      
 6   CACE cityname     141513 non-null  category      
 7   Capital           141513 non-null  category      
 8   Majorcity         141513 non-null  category      
 9   Top3cities        141513 non-null  category      
 10  adm_1             141513 non-null  category      
 11  adm_2             141513 non-null  category      
 12  latitude          141513 non-null  float64       
 13  longitude         141513 non-null  float64       
 14  country  

## 3. Análisis exploratorio de datos

In [65]:
df_cace.describe(include='category')

,active_year,type_of_violence,dyad_name,side_a,side_b,City,CACE cityname,Capital,Majorcity,Top3cities,adm_1,adm_2,country,region
count,141513,141513,141513,141513,141513,141513,141513,141513,141513,141513,141513,141513,141513,141513
unique,2,3,1239,696,710,2,981,2,2,2,1416,6425,118,5
top,1,1,Government of Afghanistan - Taleban,Government of Afghanistan,Civilians,0,,0,0,0,Jammu and Kashmir State,,Afghanistan,Asia
freq,134389,100004,21237,23807,29302,117033,117033,134420,132823,130237,7626,36735,25116,67505


En la descripción general, se pueden destacar los siguientes puntos:

- De las entidades participantes, las que tienen mas eventos son:
  - El gobierno de Afganistan (puede deberse a conflictos bélicos)
  - Civiles (atentados o daños colaterales)
- El conflicto (diada o par de entidades) que tiene mas eventos es entre el gobiernos de Afganistán y los talibanes.
- En base a lo anterior, es lógico que el país y continente con mas eventos sea Afganistan y Asia.


#### b. Entidades participantes

In [66]:
# Primeras 10 entidades que inician el evento (o atacan) que mas se repiten
df_cace['side_a'].value_counts().head(10)

side_a
Government of Afghanistan              23807
Government of India                    10257
Government of Iraq                      5414
Government of Turkey                    4538
Government of Nepal                     4490
Government of Sri Lanka                 4167
Government of Pakistan                  4031
Government of Algeria                   3778
Government of Russia (Soviet Union)     3561
Government of Colombia                  3466
Name: count, dtype: int64

In [67]:
# Primeras 10 entidades que son atacadas que mas se repiten
df_cace['side_b'].value_counts().head(10)

side_b
Civilians             29302
Taleban               21372
IS                     6470
Kashmir insurgents     5837
PKK                    4469
LTTE                   4001
CPN-M                  3669
TTP                    3553
FARC                   2864
Al-Shabaab             2817
Name: count, dtype: int64

In [68]:
# Primeros 10 par de actores enfretados en un evento
df_cace['dyad_name'].value_counts().head(10)

dyad_name
Government of Afghanistan - Taleban         21237
Government of India - Kashmir insurgents     5837
Government of Iraq - IS                      4621
Government of Turkey - PKK                   4434
Government of Sri Lanka - LTTE               3993
Government of Nepal - CPN-M                  3669
Government of Pakistan - TTP                 3489
Government of Colombia - FARC                2729
Government of Somalia - Al-Shabaab           2678
Government of Algeria - AQIM                 1816
Name: count, dtype: int64